In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
#from ccfj import GetStationPairs
from geopy.distance import great_circle
#import shapefile
import geopandas as gp
import yaml
import math

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon

from pykrige.ok import OrdinaryKriging


/tmp/ipykernel_81366/803535026.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib
from toollib_standard import mathlib

In [3]:
flag_project = 1 # 0--regular; 1--repartrition

In [4]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

#name_project = 'project/output_FJSJ_16-01/'               
#name_project = 'project_repartrition/repartrition_01-03/'               
#name_project = 'project_voronoi/voronoi_01-03/'         

In [5]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.5_02--10-16Hz/


In [6]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [7]:
dir_model_fund = dir_project + info_basic['rdir_model_fund']
dir_model = dir_project + info_basic['rdir_model']
dir_image = dir_project + info_basic['rdir_image']+'Vs_compare/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_inv = dir_project + info_basic['rdir_inv_BFGS']
dir_partition = dir_project + info_basic['rdir_partition']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

['264--12-10',
 '536--49-16',
 '63--47-04',
 '493--38-15',
 '338--46-11',
 '249--40-09',
 '210--45-08',
 '41--10-04',
 '540--07-17',
 '184--18-08',
 '411--30-13',
 '201--36-08',
 '406--25-13',
 '324--32-11',
 '15--23-03',
 '572--09-18',
 '307--08-11',
 '476--08-15',
 '55--31-04',
 '113--25-06',
 '175--51-07',
 '169--45-07',
 '385--49-12',
 '9--16-03',
 '218--53-08',
 '512--09-16',
 '183--17-08',
 '272--21-10',
 '545--13-17',
 '535--48-16',
 '56--32-04',
 '337--45-11',
 '89--40-05',
 '489--34-15',
 '575--12-18',
 '277--26-10',
 '556--35-17',
 '65--49-04',
 '10--18-03',
 '593--50-18',
 '199--34-08',
 '367--31-12',
 '390--54-12',
 '282--31-10',
 '204--39-08',
 '461--41-14',
 '423--42-13',
 '157--32-07',
 '392--09-13',
 '332--40-11',
 '442--18-14',
 '357--20-12',
 '353--15-12',
 '372--36-12',
 '233--24-09',
 '248--39-09',
 '531--44-16',
 '284--33-10',
 '191--25-08',
 '510--07-16',
 '568--53-17',
 '217--52-08',
 '36--52-03',
 '44--17-04',
 '521--34-16',
 '105--14-06',
 '437--08-14',
 '93--4

In [8]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 

In [9]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
num_stations = []
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    num_stations.append(len(stations_this))
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [10]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [11]:
def compute_affine_transform(original_points, target_points):
    A_matrix = np.array([[original_points[0][0], original_points[0][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[0][0], original_points[0][1], 1],
                         [original_points[1][0], original_points[1][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[1][0], original_points[1][1], 1],
                         [original_points[2][0], original_points[2][1], 1, 0, 0, 0],
                         [0, 0, 0, original_points[2][0], original_points[2][1], 1]])

    A1_B1_C1 = np.array([target_points[0][0], target_points[0][1], target_points[1][0], target_points[1][1], target_points[2][0], target_points[2][1]])

    coefficients = np.linalg.solve(A_matrix, A1_B1_C1)

    affine_matrix = np.array([[coefficients[0], coefficients[1], coefficients[2]],
                               [coefficients[3], coefficients[4], coefficients[5]],
                               [0, 0, 1]])

    return affine_matrix

In [12]:
# Affine transformation
lon_stations_all_new = []
lat_stations_all_new = []
for sta in stalist_all:
    if int(sta[1:3]) <= 60:
        lon_stations_all_new.append(lon_stations_all[stalist_all.index(sta)])
        lat_stations_all_new.append(lat_stations_all[stalist_all.index(sta)])
refs = ['R0101','R6001','R6020']
lon_refs = [lon_stations_all[stalist_all.index(ref)] for ref in refs]
lat_refs = [lat_stations_all[stalist_all.index(ref)] for ref in refs]
loc_refs = np.column_stack([lon_refs,lat_refs])
loc_refs_new = np.array([[0,0],[600,0],[600,600]])

affine_matrix = compute_affine_transform(loc_refs, loc_refs_new)

In [13]:
# all stations
x_stations_all_new = []
y_stations_all_new = []
for i in range(len(lon_stations_all_new)):
    loc_sta = np.array([lon_stations_all_new[i],lat_stations_all_new[i],1])
    loc_sta_new = np.dot(affine_matrix,loc_sta)
    x_stations_all_new.append(loc_sta_new[0])
    y_stations_all_new.append(loc_sta_new[1])
# faults
x_faults = {}
y_faults = {}
for i in range(len(faults)):
    x_faults['clark'+str(i+1)] = []
    y_faults['clark'+str(i+1)] = []
    for j in range(len(faults['clark'+str(i+1)]['lon'])):
        loc_fault = np.array([faults['clark'+str(i+1)]['lon'][j],faults['clark'+str(i+1)]['lat'][j],1])
        loc_fault_new = np.dot(affine_matrix,loc_fault)
        x_faults['clark'+str(i+1)].append(loc_fault_new[0])
        y_faults['clark'+str(i+1)].append(loc_fault_new[1])
# partitions
x_centroid_partition = []
y_centroid_partition = []
for i in range(len(lon_centroid_partition)):
    loc_centroid = np.array([lon_centroid_partition[i],lat_centroid_partition[i],1])
    loc_centroid_new = np.dot(affine_matrix,loc_centroid)
    x_centroid_partition.append(loc_centroid_new[0])
    y_centroid_partition.append(loc_centroid_new[1])

In [14]:
inter_file = dir_inv + 'vs_inter.npz'
inter = np.load(inter_file, allow_pickle=True)

In [15]:
vs_fund = inter['vs_fund']
vs = inter['vs']
x_inter_in = inter['x']
y_inter_in = inter['y']
z = inter['depth']
dz = inter['dz']
N = inter['N']

In [16]:
struc = {}
struc_fund = {}
flag = 0
for key_subwork in key_subworks:
    file_model = dir_model + 'model_'+key_subwork+'.txt'
    file_model_fund = dir_model_fund + 'model_'+key_subwork+'.txt'
    model = np.loadtxt(file_model)
    model_fund = np.loadtxt(file_model_fund)
    struc[key_subwork] = {}
    struc[key_subwork]['layer'] = model[:, 0]
    struc[key_subwork]['z'] = model[:, 1]
    struc[key_subwork]['rho'] = model[:, 2]
    struc[key_subwork]['vs'] = model[:, 3]
    struc[key_subwork]['vp'] = model[:, 4]
    struc[key_subwork]['std'] = model[:, 5]
    struc_fund[key_subwork] = {}
    struc_fund[key_subwork]['layer'] = model_fund[:, 0]
    struc_fund[key_subwork]['z'] = model_fund[:, 1]
    struc_fund[key_subwork]['rho'] = model_fund[:, 2]
    struc_fund[key_subwork]['vs'] = model_fund[:, 3]
    struc_fund[key_subwork]['vp'] = model_fund[:, 4]
    struc_fund[key_subwork]['std'] = model_fund[:, 5]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))

vs_horizon = {}
vs_horizon_fund = {}
for i in range(N):
    vs_horizon[i] = []
    vs_horizon_fund[i] = []
    for key_subwork in key_subworks:
        vs_horizon[i].append(struc[key_subwork]['vs'][i])
        vs_horizon_fund[i].append(struc_fund[key_subwork]['vs'][i])

Read structure model: 264--12-10 (1/326)
Read structure model: 536--49-16 (2/326)
Read structure model: 63--47-04 (3/326)
Read structure model: 493--38-15 (4/326)
Read structure model: 338--46-11 (5/326)
Read structure model: 249--40-09 (6/326)
Read structure model: 210--45-08 (7/326)
Read structure model: 41--10-04 (8/326)
Read structure model: 540--07-17 (9/326)
Read structure model: 184--18-08 (10/326)
Read structure model: 411--30-13 (11/326)
Read structure model: 201--36-08 (12/326)
Read structure model: 406--25-13 (13/326)
Read structure model: 324--32-11 (14/326)
Read structure model: 15--23-03 (15/326)
Read structure model: 572--09-18 (16/326)
Read structure model: 307--08-11 (17/326)
Read structure model: 476--08-15 (18/326)
Read structure model: 55--31-04 (19/326)
Read structure model: 113--25-06 (20/326)
Read structure model: 175--51-07 (21/326)
Read structure model: 169--45-07 (22/326)
Read structure model: 385--49-12 (23/326)
Read structure model: 9--16-03 (24/326)
Read st

In [17]:
for i in range(N):
    print('Plot horizon %d/%d' % (i+1,N))
    fig = plt.figure(figsize=(20, 14))
    ax = fig.add_subplot(221)
    ax.scatter(x_stations_all_new, y_stations_all_new,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    flag_add = 0.0005
    ax.set_xlim(-50,650)
    ax.set_ylim(-50,650)
    for j in range(len(faults)):
        ax.plot(x_faults['clark'+str(j+1)], y_faults['clark'+str(j+1)], 'k')
    im = ax.scatter(x_centroid_partition,y_centroid_partition,c=vs_horizon_fund[i],cmap='gist_rainbow',s=100)
    plt.colorbar(im)
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(222)
    ax.scatter(x_stations_all_new, y_stations_all_new,marker='.',color='k')
    for j in range(len(faults)):
        ax.plot(x_faults['clark'+str(j+1)], y_faults['clark'+str(j+1)], 'k')
    #im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_horizon_in[i], cmap='gist_rainbow', vmin=v_min, vmax=v_max)
    im = plt.scatter(x_inter_in, y_inter_in, s=10, c=vs_fund[i], cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    ax.set_xlim(-50,650)
    ax.set_ylim(-50,650)
    

    ax = fig.add_subplot(223)
    ax.scatter(x_stations_all_new, y_stations_all_new,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    flag_add = 0.0005
    ax.set_xlim(-50,650)
    ax.set_ylim(-50,650)
    for j in range(len(faults)):
        ax.plot(x_faults['clark'+str(j+1)], y_faults['clark'+str(j+1)], 'k')
    im = ax.scatter(x_centroid_partition,y_centroid_partition,c=vs_horizon[i],cmap='gist_rainbow',s=100)
    plt.colorbar(im)
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Overtones')
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(224)
    ax.scatter(x_stations_all_new, y_stations_all_new,marker='.',color='k')
    for j in range(len(faults)):
        ax.plot(x_faults['clark'+str(j+1)], y_faults['clark'+str(j+1)], 'k')
    #im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_horizon_in[i], cmap='gist_rainbow', vmin=v_min, vmax=v_max)
    im = plt.scatter(x_inter_in, y_inter_in, s=10, c=vs[i], cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlim(-50,650)
    ax.set_ylim(-50,650)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.tight_layout()
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Overtones')
    plt.savefig(dir_image+'Vs_hori_'+str((i+1)*dz)+'m.png',dpi=100)
    plt.close()

Plot horizon 1/40
Plot horizon 2/40
Plot horizon 3/40
Plot horizon 4/40
Plot horizon 5/40
Plot horizon 6/40
Plot horizon 7/40
Plot horizon 8/40
Plot horizon 9/40
Plot horizon 10/40
Plot horizon 11/40
Plot horizon 12/40
Plot horizon 13/40
Plot horizon 14/40
Plot horizon 15/40
Plot horizon 16/40
Plot horizon 17/40
Plot horizon 18/40
Plot horizon 19/40
Plot horizon 20/40
Plot horizon 21/40
Plot horizon 22/40
Plot horizon 23/40
Plot horizon 24/40
Plot horizon 25/40
Plot horizon 26/40
Plot horizon 27/40
Plot horizon 28/40
Plot horizon 29/40
Plot horizon 30/40
Plot horizon 31/40
Plot horizon 32/40
Plot horizon 33/40
Plot horizon 34/40
Plot horizon 35/40
Plot horizon 36/40
Plot horizon 37/40
Plot horizon 38/40
Plot horizon 39/40
Plot horizon 40/40
